# Configuration

In [1]:
import os
import sys


os.environ["SPARK_HOME"] = "/usr/spark2.4.3"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda/bin/python" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/anaconda/bin/python"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

#Importing libraries that are required

import csv
import matplotlib.pyplot as plt
import gmplot
import pandas as pd
import numpy as np
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.sql.functions import col, lit, when

#Importing SQL context

from io import StringIO
from collections import namedtuple
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import sum, col, desc

#Machine Learning library for movie recommendation

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit,ParamGridBuilder
from pyspark.sql.functions import explode

In [2]:
#New Spark Session

import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sqlContext = SQLContext(spark)

In [3]:
#Sparksession on Recommendation system

spark=SparkSession.builder.appName("Recomendation").getOrCreate()

# Creating dataframes

In [4]:
#Creating new Data Frame on Movies dataset which contains large data

Movies_Big= spark.read.option("header", "true").option("delimiter", ",")\
.option("inferSchema", "true").csv("hdfs:///user/imat5322_772599/Assessment_2/movies.csv")

In [5]:
#Performing action on the movie dataset to check if the data is uploaded successfully

Movies_Big.take(5)

[Row(movieId=1, title='Toy Story (1995)', genres='Adventure|Animation|Children|Comedy|Fantasy'),
 Row(movieId=2, title='Jumanji (1995)', genres='Adventure|Children|Fantasy'),
 Row(movieId=3, title='Grumpier Old Men (1995)', genres='Comedy|Romance'),
 Row(movieId=4, title='Waiting to Exhale (1995)', genres='Comedy|Drama|Romance'),
 Row(movieId=5, title='Father of the Bride Part II (1995)', genres='Comedy')]

In [6]:
#Creating new Data Frame on Ratings dataset which contains large data

Ratings_Big= spark.read.option("header", "true").option("delimiter", ",")\
.option("inferSchema", "true").csv("hdfs:///user/imat5322_772599/Assessment_2/ratings.csv")

In [7]:
#Performing action on the ratings dataset to check if the data is uploaded successfully

Ratings_Big.take(5)

[Row(userId=1, movieId=307, rating=3.5, timestamp=1256677221),
 Row(userId=1, movieId=481, rating=3.5, timestamp=1256677456),
 Row(userId=1, movieId=1091, rating=1.5, timestamp=1256677471),
 Row(userId=1, movieId=1257, rating=4.5, timestamp=1256677460),
 Row(userId=1, movieId=1449, rating=4.5, timestamp=1256677264)]

In [8]:
#Checking the count of the observations of Movies_Big Data Frame

Movies_Big.count()

58098

In [9]:
#Checking the count of the observations of Ratings_Big Data Frame

Ratings_Big.count()

27753444

In [10]:
#Checking the Schema of the Movie_Big Data Frames

Movies_Big.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [11]:
#Checking the Schema of the Ratings_Big Data Frames

Ratings_Big.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



# Data Exploration

In [12]:
#Checking for missing data on each column
#Counter is set to 0

i = 0 

#Total saves the sum of the missing data in Movie_Big

total = Movies_Big.count()

print("Total Records = " + str(total))

#Prints the total number of column

print("Total columns = " + str(len(Movies_Big.columns)))
print("------------------------------------------")

#Loop is used to display the count of missing values of each column

#Loop checking every column

while i < len(Movies_Big.columns):                            
    print(str(i+1) + "." + str(Movies_Big[i]))
    print("  Missing Values = ") 
   
    missingvalue = Movies_Big.select([count(when(Movies_Big[i].isNull(),\
                                        True))]).show() 
    
    #Counter is incremented by 1
    i = i+1 

Total Records = 58098
Total columns = 3
------------------------------------------
1.Column<b'movieId'>
  Missing Values = 
+------------------------------------------------+
|count(CASE WHEN (movieId IS NULL) THEN true END)|
+------------------------------------------------+
|                                               0|
+------------------------------------------------+

2.Column<b'title'>
  Missing Values = 
+----------------------------------------------+
|count(CASE WHEN (title IS NULL) THEN true END)|
+----------------------------------------------+
|                                             0|
+----------------------------------------------+

3.Column<b'genres'>
  Missing Values = 
+-----------------------------------------------+
|count(CASE WHEN (genres IS NULL) THEN true END)|
+-----------------------------------------------+
|                                              0|
+-----------------------------------------------+



In [13]:
#Checking for missing data on each column
#Counter is set to 0

i = 0 

#Total saves the sum of the Ratings_Big

total = Ratings_Big.count()

print("Total Records = " + str(total))

#Prints the total number of column

print("Total columns = " + str(len(Ratings_Big.columns)))
print("------------------------------------------")

#Loop is used to display the count of missing values of each column
#Loop checking every column

while i < len(Ratings_Big.columns):                             
    print(str(i+1) + "." + str(Ratings_Big[i]))
    print("  Missing Values = ") 
   
    missingvalue = Ratings_Big.select([count(when(Ratings_Big[i].isNull(),\
                                        True))]).show() 
    
    #Counter is incremented by 1
    i = i+1 

Total Records = 27753444
Total columns = 4
------------------------------------------
1.Column<b'userId'>
  Missing Values = 
+-----------------------------------------------+
|count(CASE WHEN (userId IS NULL) THEN true END)|
+-----------------------------------------------+
|                                              0|
+-----------------------------------------------+

2.Column<b'movieId'>
  Missing Values = 
+------------------------------------------------+
|count(CASE WHEN (movieId IS NULL) THEN true END)|
+------------------------------------------------+
|                                               0|
+------------------------------------------------+

3.Column<b'rating'>
  Missing Values = 
+-----------------------------------------------+
|count(CASE WHEN (rating IS NULL) THEN true END)|
+-----------------------------------------------+
|                                              0|
+-----------------------------------------------+

4.Column<b'timestamp'>
  Missing Va

# Cleaning of data

In [14]:
#Dropping timestamp column as it is not needed for analysis 

Ratings_Big=Ratings_Big.drop("timestamp")

# Pre Processing of Data Frame

## Pre Processing on Movies_Big Data Frame

In [15]:
#Splitting the title to extarct Year using substring

year1=Movies_Big.selectExpr('title', 'substring(title, length(title)-4, length(title)) as Year')
year2= year1.withColumn("Year",expr("substring(Year, 1, length(Year)-1)"))

#Converting string type of year to integer

year = year2.withColumn("Year", year2.Year.cast('integer'))

In [16]:
#Displaying the Year column extracted from Title

year.show()

+--------------------+----+
|               title|Year|
+--------------------+----+
|    Toy Story (1995)|1995|
|      Jumanji (1995)|1995|
|Grumpier Old Men ...|1995|
|Waiting to Exhale...|1995|
|Father of the Bri...|1995|
|         Heat (1995)|1995|
|      Sabrina (1995)|1995|
| Tom and Huck (1995)|1995|
| Sudden Death (1995)|1995|
|    GoldenEye (1995)|1995|
|American Presiden...|1995|
|Dracula: Dead and...|1995|
|        Balto (1995)|1995|
|        Nixon (1995)|1995|
|Cutthroat Island ...|1995|
|       Casino (1995)|1995|
|Sense and Sensibi...|1995|
|   Four Rooms (1995)|1995|
|Ace Ventura: When...|1995|
|  Money Train (1995)|1995|
+--------------------+----+
only showing top 20 rows



In [17]:
#Schema check and Year was string type but now its changed to integer

year.printSchema()

root
 |-- title: string (nullable = true)
 |-- Year: integer (nullable = true)



In [18]:
#Merging Movies_Big with Year

Updated_Movies = Movies_Big.join(year, \
Movies_Big["title"] == year["title"], \
how="inner").drop(Movies_Big["title"])

In [19]:
Updated_Movies.show()

+-------+--------------------+--------------------+----+
|movieId|              genres|               title|Year|
+-------+--------------------+--------------------+----+
|      1|Adventure|Animati...|    Toy Story (1995)|1995|
|      2|Adventure|Childre...|      Jumanji (1995)|1995|
|      3|      Comedy|Romance|Grumpier Old Men ...|1995|
|      4|Comedy|Drama|Romance|Waiting to Exhale...|1995|
|      5|              Comedy|Father of the Bri...|1995|
|      6|Action|Crime|Thri...|         Heat (1995)|1995|
|      7|      Comedy|Romance|      Sabrina (1995)|1995|
|      8|  Adventure|Children| Tom and Huck (1995)|1995|
|      9|              Action| Sudden Death (1995)|1995|
|     10|Action|Adventure|...|    GoldenEye (1995)|1995|
|     11|Comedy|Drama|Romance|American Presiden...|1995|
|     12|       Comedy|Horror|Dracula: Dead and...|1995|
|     13|Adventure|Animati...|        Balto (1995)|1995|
|     14|               Drama|        Nixon (1995)|1995|
|     15|Action|Adventure|...|C

## Pre Processing on Ratings_Big Data Frame

In [20]:
#Extracting the average rating and count of the rating 

Ratings=Ratings_Big.groupby("movieId") \
       .agg(mean("rating").alias("avg_rating"), \
       count("userId").alias("rating_count")) \
       .sort(asc("movieId"))

In [21]:
Ratings.show()

+-------+------------------+------------+
|movieId|        avg_rating|rating_count|
+-------+------------------+------------+
|      1|3.8866494325899312|       68469|
|      2| 3.246582912721512|       27143|
|      3| 3.173981392364453|       15585|
|      4| 2.874539979926397|        2989|
|      5|3.0772909396406876|       15474|
|      6|3.8442108566049575|       28683|
|      7| 3.371348277890334|       15301|
|      8| 3.122482131254061|        1539|
|      9| 3.007529781973477|        4449|
|     10| 3.431632714743396|       33086|
|     11| 3.660277594183741|       19669|
|     12|2.6696507515473034|        4524|
|     13|3.3396516393442623|        1952|
|     14|3.4289997075168177|        6838|
|     15| 2.730976537729867|        3154|
|     16|3.8023623907394284|       21165|
|     17|3.9501262626262625|       24552|
|     18|  3.41207034372502|        6255|
|     19|2.6420142094488823|       24913|
|     20| 2.894482610562473|        4658|
+-------+------------------+------

## Merging of Data Frames

In [22]:
#Merging Updated_Movies with Ratings Data Frame

Movie_Rating = Updated_Movies.join(Ratings, \
Updated_Movies["movieId"] == Ratings["movieId"], \
how="inner").drop(Updated_Movies["movieId"]).sort(asc("movieId"))

In [23]:
Movie_Rating.show()

+--------------------+--------------------+----+-------+------------------+------------+
|              genres|               title|Year|movieId|        avg_rating|rating_count|
+--------------------+--------------------+----+-------+------------------+------------+
|Adventure|Animati...|    Toy Story (1995)|1995|      1|3.8866494325899312|       68469|
|Adventure|Childre...|      Jumanji (1995)|1995|      2| 3.246582912721512|       27143|
|      Comedy|Romance|Grumpier Old Men ...|1995|      3| 3.173981392364453|       15585|
|Comedy|Drama|Romance|Waiting to Exhale...|1995|      4| 2.874539979926397|        2989|
|              Comedy|Father of the Bri...|1995|      5|3.0772909396406876|       15474|
|Action|Crime|Thri...|         Heat (1995)|1995|      6|3.8442108566049575|       28683|
|      Comedy|Romance|      Sabrina (1995)|1995|      7| 3.371348277890334|       15301|
|  Adventure|Children| Tom and Huck (1995)|1995|      8| 3.122482131254061|        1539|
|              Action

In [24]:
Movie_Rating.count()

54039

In [25]:
#Rearranging the column

Movie_Rating = Movie_Rating \
.select("movieId","title","Year","genres","avg_rating","rating_count")

In [26]:
#Removing year from title column

Movie_Rating = Movie_Rating \
.withColumn("title",expr("substring(title, 1, length(title)-7)"))

In [27]:
Movie_Rating.show()

+-------+--------------------+----+--------------------+------------------+------------+
|movieId|               title|Year|              genres|        avg_rating|rating_count|
+-------+--------------------+----+--------------------+------------------+------------+
|      1|           Toy Story|1995|Adventure|Animati...|3.8866494325899312|       68469|
|      2|             Jumanji|1995|Adventure|Childre...| 3.246582912721512|       27143|
|      3|    Grumpier Old Men|1995|      Comedy|Romance| 3.173981392364453|       15585|
|      4|   Waiting to Exhale|1995|Comedy|Drama|Romance| 2.874539979926397|        2989|
|      5|Father of the Bri...|1995|              Comedy|3.0772909396406876|       15474|
|      6|                Heat|1995|Action|Crime|Thri...|3.8442108566049575|       28683|
|      7|             Sabrina|1995|      Comedy|Romance| 3.371348277890334|       15301|
|      8|        Tom and Huck|1995|  Adventure|Children| 3.122482131254061|        1539|
|      9|        Sudd

## Extracting Genres individually

In [28]:
#Splitting multiple genres using delimiter '|'

from pyspark.sql.functions import split,explode
Genres=Updated_Movies.withColumn('genres',explode(split('genres','[|]')))

In [29]:
Genres.show()

+-------+---------+--------------------+----+
|movieId|   genres|               title|Year|
+-------+---------+--------------------+----+
|      1|Adventure|    Toy Story (1995)|1995|
|      1|Animation|    Toy Story (1995)|1995|
|      1| Children|    Toy Story (1995)|1995|
|      1|   Comedy|    Toy Story (1995)|1995|
|      1|  Fantasy|    Toy Story (1995)|1995|
|      2|Adventure|      Jumanji (1995)|1995|
|      2| Children|      Jumanji (1995)|1995|
|      2|  Fantasy|      Jumanji (1995)|1995|
|      3|   Comedy|Grumpier Old Men ...|1995|
|      3|  Romance|Grumpier Old Men ...|1995|
|      4|   Comedy|Waiting to Exhale...|1995|
|      4|    Drama|Waiting to Exhale...|1995|
|      4|  Romance|Waiting to Exhale...|1995|
|      5|   Comedy|Father of the Bri...|1995|
|      6|   Action|         Heat (1995)|1995|
|      6|    Crime|         Heat (1995)|1995|
|      6| Thriller|         Heat (1995)|1995|
|      7|   Comedy|      Sabrina (1995)|1995|
|      7|  Romance|      Sabrina (

In [30]:
Genres.groupBy("genres").count().show()

+------------------+-----+
|            genres|count|
+------------------+-----+
|             Crime| 5117|
|           Romance| 7434|
|          Thriller| 8260|
|         Adventure| 4081|
|             Drama|24217|
|               War| 1821|
|       Documentary| 5124|
|           Fantasy| 2641|
|           Mystery| 2776|
|           Musical| 1115|
|         Animation| 2670|
|         Film-Noir|  364|
|(no genres listed)| 4278|
|              IMAX|  197|
|            Horror| 5578|
|           Western| 1379|
|            Comedy|15989|
|          Children| 2753|
|            Action| 7148|
|            Sci-Fi| 3462|
+------------------+-----+



# Visualisation

In [31]:
#Importing PixieDust for visualisation

import pixiedust

Pixiedust database opened successfully


## Analysis on Average Rating vs Total Rating

In [32]:
display(Movie_Rating)

## Analysis on Individual Genre

In [33]:
display(Genres)

## Analysis on Top movies whose total rating is above 70,000

In [34]:
#Creating a temporary table

Topmovies=Movie_Rating.registerTempTable("Topmovies")

In [35]:
#Extracting movies with total rating over 70,000

Topmovies=sqlContext.\
sql("select * \
from Topmovies \
where rating_count > 70000 \
order by rating_count")

In [36]:
Topmovies.show()

+-------+--------------------+----+--------------------+------------------+------------+
|movieId|               title|Year|              genres|        avg_rating|rating_count|
+-------+--------------------+----+--------------------+------------------+------------+
|    527|    Schindler's List|1993|           Drama|War| 4.257501817775044|       71516|
|    480|       Jurassic Park|1993|Action|Adventure|...|3.6650338125073576|       76451|
|    260|Star Wars: Episod...|1977|Action|Adventure|...| 4.120454684348836|       81815|
|   2571|         Matrix, The|1999|Action|Sci-Fi|Thr...| 4.149695428470046|       84545|
|    593|Silence of the La...|1991|Crime|Horror|Thri...|  4.15141241652351|       87899|
|    296|        Pulp Fiction|1994|Comedy|Crime|Dram...| 4.173971387139363|       92406|
|    356|        Forrest Gump|1994|Comedy|Drama|Roma...| 4.056584913437757|       97040|
|    318|Shawshank Redempt...|1994|         Crime|Drama| 4.424188001918387|       97999|
+-------+------------

In [37]:
display(Topmovies)

## Visualisation on the trend of movies produced in each year

In [38]:
Movie_Rating.select("Year")\
.groupby("Year").count().orderBy(desc("count")).show()

+----+-----+
|Year|count|
+----+-----+
|2015| 2244|
|2014| 2221|
|2016| 2163|
|2013| 2107|
|2012| 1940|
|2011| 1833|
|2017| 1832|
|2009| 1752|
|2010| 1681|
|2008| 1610|
|2007| 1458|
|2006| 1406|
|2005| 1226|
|2004| 1127|
|2003| 1000|
|2002|  996|
|2001|  937|
|2000|  907|
|2018|  826|
|1999|  788|
+----+-----+
only showing top 20 rows



In [39]:
Year_Count= Movie_Rating.select(Movie_Rating.Year).groupby(Movie_Rating.Year).count()

In [40]:
Year_Count.registerTempTable("yearwise")

In [41]:
Year_Wise_Count = sqlContext.\
sql("select Year, \
SUM(count) as Total \
 from yearwise group by Year order by Year")

In [42]:
Year_Wise_Count.show()

+----+-----+
|Year|Total|
+----+-----+
|null|  499|
|   6|    1|
|1874|    1|
|1878|    1|
|1883|    1|
|1887|    1|
|1888|    4|
|1890|    5|
|1891|    6|
|1892|    3|
|1893|    1|
|1894|   21|
|1895|    8|
|1896|   19|
|1897|   11|
|1898|   12|
|1899|    6|
|1900|   17|
|1901|   13|
|1902|    4|
+----+-----+
only showing top 20 rows



In [43]:
display(Year_Wise_Count)

In [ ]:
#Displaying the years with movie count more than 1000

display(Year_Wise_Count)

In [45]:
#Displaying the years with movie count less than 10

display(Year_Wise_Count)

## Comparision of Genre combination between 20th Century and 21st Century

In [46]:
Nintees_Movies=Movie_Rating.registerTempTable("Nintees_Movies")

In [47]:
#extracting movies which is releases before 2000
Nintees_Movies=sqlContext.\
sql("select * \
from Nintees_Movies \
where Year < 2000 and Year >1900 \
order by rating_count")

In [48]:
Nintees_Movies.show()

+-------+--------------------+----+--------------------+----------+------------+
|movieId|               title|Year|              genres|avg_rating|rating_count|
+-------+--------------------+----+--------------------+----------+------------+
| 120749|Mexican Spitfire ...|1940|              Comedy|       2.0|           1|
|  90493|War of the Robots...|1978|Action|Adventure|...|       2.5|           1|
| 113812|        Bullet Scars|1942|         Crime|Drama|       2.5|           1|
|  96706|   Straight Shooting|1917|             Western|       4.0|           1|
| 112251|            Snapshot|1979|Drama|Horror|Thri...|       1.0|           1|
| 127092|The Kiss of Her F...|1968|     Horror|Thriller|       1.0|           1|
| 168902| Behind Green Lights|1946|               Crime|       3.5|           1|
| 134873|              Ginger|1971|  Action|Crime|Drama|       2.0|           1|
|  93905|    The Gamma People|1956|       Horror|Sci-Fi|       2.0|           1|
| 126309|     Vessel of Wrat

In [49]:
display(Nintees_Movies)

In [50]:
Two_Thousands_Movies=Movie_Rating.registerTempTable("Two_Thousands_Movies")

In [51]:
#extracting movies which is releases after 2000
Two_Thousands_Movies=sqlContext.\
sql("select * \
from Two_Thousands_Movies \
where Year > 1999  \
order by rating_count")

In [52]:
Two_Thousands_Movies.show()

+-------+--------------------+----+--------------------+----------+------------+
|movieId|               title|Year|              genres|avg_rating|rating_count|
+-------+--------------------+----+--------------------+----------+------------+
|  89537| Unlikely Weapon, An|2008|         Documentary|       3.5|           1|
| 121763|Without Lying Dow...|2000|         Documentary|       3.0|           1|
| 136924|Tekken: Kazuya's ...|2014|              Action|       2.5|           1|
| 165495|            Ladrones|2015|        Comedy|Crime|       2.5|           1|
| 163046|            Hattrick|2007|  (no genres listed)|       3.0|           1|
| 145038|            Devotion|2003|               Drama|       4.0|           1|
| 148324|              Reaper|2000|Horror|Mystery|Th...|       1.5|           1|
| 149764|The Art of Seduction|2005|Comedy|Drama|Romance|       1.5|           1|
| 154202|      Trench of Hope|2003|               Drama|       2.0|           1|
| 162260|     Deuce of Spade

In [53]:
display(Two_Thousands_Movies)

# Movie Recommendation System

In [54]:
#CloudxLab cant handle huge data for recommendation 

#So,the datasets with less observations are taken for Recommendation system
#The datasets are taken from same MovieLens website
#Reference: https://www.youtube.com/watch?v=rgb6Yy4IGpk&t=485s

## Creating Data Frames for Movies and Ratings with Small data

In [55]:
Ratings_Small= spark.read.option("header", "true").option("delimiter", ",")\
.option("inferSchema", "true").csv("hdfs:///user/imat5322_772599/movies/ratings_small.csv")

In [56]:
Ratings_Small.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [57]:
Movies_Small= spark.read.option("header", "true").option("delimiter", ",")\
.option("inferSchema", "true").csv("hdfs:///user/imat5322_772599/movies/movies_small.csv")

In [58]:
Movies_Small.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

### Cleaning of Data Frame

In [59]:
Ratings_Small.drop('timestamp')

DataFrame[userId: int, movieId: int, rating: double]

## Configuration for Machine Learning

In [60]:
#Importing machine learning collaborating filtereing Alternative Least Squares

In [61]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

## Splitting the datasets into train and test

In [62]:
(train,test)=Ratings_Small.randomSplit([0.6,0.4], seed=42)

In [63]:
train.take(3)

[Row(userId=1, movieId=50, rating=5.0, timestamp=964982931),
 Row(userId=1, movieId=70, rating=3.0, timestamp=964982400),
 Row(userId=1, movieId=101, rating=5.0, timestamp=964980868)]

In [64]:
test.take(3)

[Row(userId=1, movieId=1, rating=4.0, timestamp=964982703),
 Row(userId=1, movieId=3, rating=4.0, timestamp=964981247),
 Row(userId=1, movieId=6, rating=4.0, timestamp=964982224)]

In [65]:
als=ALS(maxIter=5,regParam=0.01,userCol="userId",itemCol="movieId",ratingCol="rating",coldStartStrategy="drop",nonnegative= True)

In [66]:
#Model creation

model=als.fit(train)

In [67]:
predictions=model.transform(test)

In [68]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")

In [69]:
#Root Mean Square Error

rmse=evaluator.evaluate(predictions)

In [70]:
print("RMSE="+str(rmse))

RMSE=1.0492045710797915


In [71]:
als=ALS(userCol="userId",itemCol="movieId",ratingCol="rating",coldStartStrategy="drop",nonnegative=True)

In [72]:
#Creates the matrix

param_grid=ParamGridBuilder().addGrid(als.rank,[5,10,15]).addGrid(als.maxIter,[5,10,15]).addGrid(als.regParam,[0.01,0.05,0.1]).build()

In [73]:
display(param_grid)

In [74]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")

In [75]:
display(evaluator)

RegressionEvaluator_f06d4c5cc968

In [76]:
Train_Validate_Split=TrainValidationSplit(estimator=als,estimatorParamMaps=param_grid,evaluator=evaluator)


In [77]:
model=Train_Validate_Split.fit(train)

In [78]:
Best_Model=model.bestModel
predictions=Best_Model.transform(test)
rmse=evaluator.evaluate(predictions)

In [79]:
print("RMSE="+str(rmse))
print("Rank="+str(Best_Model.rank))
print("MaxIter="+str(Best_Model._java_obj.parent().getMaxIter()))
print("RegParam="+str(Best_Model._java_obj.parent().getRegParam()))

RMSE=0.9069665545520337
Rank=10
MaxIter=10
RegParam=0.1


In [80]:
User=Best_Model.recommendForAllUsers(10).sort("userId")


In [ ]:
display(User)

In [82]:
predictions.sort("userId","rating").show()

+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|     1|   2253|   2.0|964981775| 2.7571738|
|     1|   1219|   2.0|964983393| 4.7502465|
|     1|   2389|   2.0|964983094| 2.1207445|
|     1|   2617|   2.0|964982588| 3.9391813|
|     1|   2093|   3.0|964981710| 3.9147518|
|     1|   3247|   3.0|964983108| 3.4347394|
|     1|   2657|   3.0|964983426| 3.7289064|
|     1|   1030|   3.0|964982903|  3.333755|
|     1|    296|   3.0|964982967|  4.679693|
|     1|   3243|   3.0|964981093| 3.0422611|
|     1|    316|   3.0|964982310| 3.7368307|
|     1|   2414|   3.0|964982513| 3.9527335|
|     1|   1377|   3.0|964982653| 3.6174746|
|     1|   1676|   3.0|964982620| 3.7719731|
|     1|   2640|   4.0|964982377|  4.151831|
|     1|   2096|   4.0|964982838|  4.060938|
|     1|    552|   4.0|964982653|  3.936318|
|     1|   3440|   4.0|964981799|  2.723496|
|     1|    590|   4.0|964982546| 4.4870934|
|     1|  

In [83]:
#Creating a function

def get_rec(rec):
  rec=rec.select("userId",explode(rec.recommendations))
  rec=rec.select("col.*","*")
  Func_Rating=rec.select(["userId","movieId","rating"])
  return Func_Rating

In [84]:
Func_Rating=get_rec(Best_Model.recommendForAllUsers(10))

In [85]:
Func_Rating.show()

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|   471| 177593|4.9599953|
|   471|   2511|4.9179683|
|   471|   2936|4.8650985|
|   471|  26258| 4.849306|
|   471|   1104| 4.781372|
|   471|   2351| 4.748386|
|   471|  33779| 4.729414|
|   471|  92494| 4.729414|
|   471|   3083| 4.702852|
|   471|    334| 4.686756|
|   463|   7842| 4.994829|
|   463|   3030| 4.887071|
|   463| 171495| 4.823522|
|   463|    441|4.8196015|
|   463| 175303|  4.79199|
|   463|    951|4.7730403|
|   463|   5952|4.7091293|
|   463| 115713|4.6827116|
|   463|  26133| 4.680236|
|   463|   6300|4.6791954|
+------+-------+---------+
only showing top 20 rows



## Movies Watched

In [86]:
Movies_Watched=Func_Rating.join(Movies_Small,['movieId'],how='left')

In [87]:
Movies_Watched.filter("userId=20").orderBy(Movies_Watched["rating"].desc()).show(10)

+-------+------+---------+--------------------+------------------+
|movieId|userId|   rating|               title|            genres|
+-------+------+---------+--------------------+------------------+
|   5915|    20| 5.558968|Victory (a.k.a. E...|  Action|Drama|War|
| 132333|    20| 5.558968|         Seve (2014)| Documentary|Drama|
|   5490|    20| 5.558968|  The Big Bus (1976)|     Action|Comedy|
| 177593|    20| 5.516726|Three Billboards ...|       Crime|Drama|
|   1949|    20|5.4781146|Man for All Seaso...|             Drama|
|   4429|    20| 5.452118|    Moby Dick (1956)|             Drama|
|   6375|    20| 5.383952|Gigantic (A Tale ...|       Documentary|
|   2202|    20| 5.322255|     Lifeboat (1944)|         Drama|War|
|  92494|    20|5.2772484|Dylan Moran: Mons...|Comedy|Documentary|
|  33779|    20|5.2772484|Eddie Izzard: Dre...|            Comedy|
+-------+------+---------+--------------------+------------------+



## Movies Recommended

In [88]:
Movies_Recommended=Movies_Small.join(Ratings_Small,['movieId'],how='left')
Movies_Recommended.filter("userId=20").orderBy(Movies_Recommended["rating"].desc()).show(10)

+-------+--------------------+--------------------+------+------+----------+
|movieId|               title|              genres|userId|rating| timestamp|
+-------+--------------------+--------------------+------+------+----------+
|    919|Wizard of Oz, The...|Adventure|Childre...|    20|   5.0|1054038078|
|   2083|Muppet Christmas ...|Children|Comedy|M...|    20|   5.0|1054038103|
|   1025|Sword in the Ston...|Animation|Childre...|    20|   5.0|1054038107|
|    720|Wallace & Gromit:...|Adventure|Animati...|    20|   5.0|1054038502|
|   1073|Willy Wonka & the...|Children|Comedy|F...|    20|   5.0|1054038071|
|    551|Nightmare Before ...|Animation|Childre...|    20|   5.0|1054038128|
|   1097|E.T. the Extra-Te...|Children|Drama|Sc...|    20|   5.0|1054038065|
|    594|Snow White and th...|Animation|Childre...|    20|   5.0|1054038083|
|   1148|Wallace & Gromit:...|Animation|Childre...|    20|   5.0|1054038506|
|    661|James and the Gia...|Adventure|Animati...|    20|   5.0|1054038185|